In [1]:
import numpy as np
import pandas as pd
import csv
import time
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import seaborn
import nltk
import wordcloud

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pyarrow

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
conf = SparkConf().setMaster("local[*]") \
    .setAll([("spark.driver.memory", "16g"),\
            ("spark.executor.memory", "8g"), \
            ("spark.driver.maxResultSize", "16g")])
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
#spark.sparkContext.stop()
#sc.getConf().getAll()

In [2]:
def getElapsedTime(startTime, endTime):
    elapsedTime = endTime - startTime
    return("Process time = %.2f seconds."%(elapsedTime))

In [3]:
def savePandasDFToParquet(csvPath, parqPath, rawSchema, printTime=False):
    startTime = time.time()
    pandasDF = pd.read_csv(csvPath, header=None)
    pandasDF.columns = rawSchema.names
    pandasDF.to_parquet(parqPath, engine="pyarrow")
    endTime = time.time()
    if printTime:
        print(getElapsedTime(startTime=startTime, endTime=endTime))
    return


In [4]:
def readSparkDFFromParquet(csvPath, parqPath, rawSchema, printTime=False):
    parquetFile = Path(parqPath)
    if (parquetFile.is_file() == False):
        print("Parquet file not found... converting %s to parquet!"%(csvPath))
        savePandasDFToParquet(csvPath=csvPath, parqPath=parqPath, rawSchema=rawSchema, printTime=printTime)
    sparkDF = spark.read.parquet(parqPath)
    return (sparkDF)


In [5]:
rawSchema = StructType([
    StructField("rating", IntegerType(), True),
    StructField("review_heading", StringType(), True),
    StructField("review_text", StringType(), True)
    ])

In [6]:
trainDF = readSparkDFFromParquet(csvPath="data/train.csv", parqPath="data/train.parquet", rawSchema=rawSchema, printTime=True)
testDF = readSparkDFFromParquet(csvPath="data/test.csv", parqPath="data/test.parquet", rawSchema=rawSchema, printTime=True)
trainDF.show(5)
print(trainDF.take(1)[0]["review_text"])

Parquet file not found... converting data/train.csv to parquet!
Process time = 26.17 seconds.
Parquet file not found... converting data/test.csv to parquet!
Process time = 5.56 seconds.
+------+--------------------+--------------------+
|rating|      review_heading|         review_text|
+------+--------------------+--------------------+
|     3|  more like funchuck|Gave this to my d...|
|     5|           Inspiring|I hope a lot of p...|
|     5|The best soundtra...|I'm reading a lot...|
|     4|    Chrono Cross OST|The music of Yasu...|
|     5| Too good to be true|Probably the grea...|
+------+--------------------+--------------------+
only showing top 5 rows

Gave this to my dad for a gag gift after directing "Nunsense," he got a reall kick out of it!
